## Deliverable 2. Create a Customer Travel Destinations Map.

In [78]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np
# Import API key
from config import g_key
from config import weather_api_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [68]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Atuona,PF,-9.8000,-139.0333,78.75,71,11,15.59,few clouds
1,1,Albany,US,42.6001,-73.9662,69.26,92,100,4.09,overcast clouds
2,2,Saint-Philippe,RE,-21.3585,55.7679,67.59,87,78,11.16,light rain
3,3,New Norfolk,AU,-42.7826,147.0587,53.10,49,78,11.65,broken clouds
4,4,Aklavik,CA,68.2191,-135.0107,32.23,99,100,7.49,overcast clouds
5,5,Fortuna Foothills,US,32.6578,-114.4119,106.21,4,0,6.91,clear sky
6,6,Busselton,AU,-33.6500,115.3333,57.27,70,95,8.90,overcast clouds
7,7,Mataura,NZ,-46.1927,168.8643,51.48,61,3,2.48,clear sky
8,8,Plettenberg Bay,ZA,-34.0527,23.3716,61.21,52,0,4.88,clear sky
9,9,Vardo,NO,70.3705,31.1107,37.38,81,100,9.22,overcast clouds


In [69]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
max_temp = float(input("What is the maximum temperature? "))
min_temp = float(input("What is the minimum temperature? "))

What is the maximum temperature? 85
What is the minimum temperature? 65


In [71]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] >= min_temp) & (city_data_df["Max Temp"] <= max_temp)]

In [72]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                4
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [73]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df = clean_df.dropna()
clean_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Atuona,PF,-9.8000,-139.0333,78.75,71,11,15.59,few clouds
1,1,Albany,US,42.6001,-73.9662,69.26,92,100,4.09,overcast clouds
2,2,Saint-Philippe,RE,-21.3585,55.7679,67.59,87,78,11.16,light rain
11,11,Arraial Do Cabo,BR,-22.9661,-42.0278,73.72,84,8,19.37,clear sky
12,12,Bilma,NE,18.6853,12.9164,78.71,19,0,12.44,clear sky
...,...,...,...,...,...,...,...,...,...,...
662,662,Arlit,NE,18.7369,7.3853,79.95,17,1,3.62,clear sky
670,670,Lazaro Cardenas,MX,17.9583,-102.2000,81.23,61,64,8.90,broken clouds
671,671,Pimenta Bueno,BR,-11.6725,-61.1936,75.47,88,76,2.46,broken clouds
672,672,Dakar,SN,14.6937,-17.4441,77.88,73,80,7.92,broken clouds


In [74]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Atuona,PF,78.75,few clouds,-9.8000,-139.0333,
1,Albany,US,69.26,overcast clouds,42.6001,-73.9662,
2,Saint-Philippe,RE,67.59,light rain,-21.3585,55.7679,
11,Arraial Do Cabo,BR,73.72,clear sky,-22.9661,-42.0278,
12,Bilma,NE,78.71,clear sky,18.6853,12.9164,
13,Sao Filipe,CV,75.34,overcast clouds,14.8961,-24.4956,
14,Kapaa,US,82.38,clear sky,22.0752,-159.3190,
18,Ahipara,NZ,68.40,clear sky,-35.1667,173.1667,
20,Rikitea,PF,77.04,scattered clouds,-23.1203,-134.9692,
21,Iracoubo,GF,77.77,moderate rain,5.4802,-53.2011,


In [92]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    ""
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row['Lat']
    lng = row['Lng']
    params['location'] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels['results'][0]['name']
    except (IndexError):
        print("No Hotel Found. Skipping...")

No Hotel Found. Skipping...
No Hotel Found. Skipping...
No Hotel Found. Skipping...
No Hotel Found. Skipping...
No Hotel Found. Skipping...
No Hotel Found. Skipping...
No Hotel Found. Skipping...
No Hotel Found. Skipping...
No Hotel Found. Skipping...
No Hotel Found. Skipping...
No Hotel Found. Skipping...
No Hotel Found. Skipping...
No Hotel Found. Skipping...
No Hotel Found. Skipping...
No Hotel Found. Skipping...
No Hotel Found. Skipping...
No Hotel Found. Skipping...
No Hotel Found. Skipping...
No Hotel Found. Skipping...
No Hotel Found. Skipping...


In [100]:
# 7. Drop the rows where there is no Hotel Name.
clean_df = hotel_df.replace('', np.nan, regex=True)
clean_hotel_df = clean_df.dropna()
clean_hotel_df.head(10)


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Atuona,PF,78.75,few clouds,-9.8000,-139.0333,Villa Enata
2,Saint-Philippe,RE,67.59,light rain,-21.3585,55.7679,"Chambres d'hôte ""La Trinité"""
11,Arraial Do Cabo,BR,73.72,clear sky,-22.9661,-42.0278,Pousada Porto Praia
12,Bilma,NE,78.71,clear sky,18.6853,12.9164,Expeditions Ténére voyages
13,Sao Filipe,CV,75.34,overcast clouds,14.8961,-24.4956,Hotel Xaguate
14,Kapaa,US,82.38,clear sky,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
18,Ahipara,NZ,68.40,clear sky,-35.1667,173.1667,Ahipara Holiday Park
20,Rikitea,PF,77.04,scattered clouds,-23.1203,-134.9692,People ThankYou
21,Iracoubo,GF,77.77,moderate rain,5.4802,-53.2011,Le Gîte d’Iracoubo
23,Makakilo City,US,79.12,light intensity drizzle,21.3469,-158.0858,Marriott's Ko Olina Beach Club


In [102]:
# 8a. Create the output File (CSV)
output_data_file = '../Vacation_Search/WeatherPy_Vacation.csv'
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [117]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather<dt><dd>{Current Description} and {Max Temp}</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [119]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content = hotel_info)
# 11b. Display the figure
fig = gmaps.figure(center=(30,31), zoom_level = 1.5)
fig.add_layer(marker_layer)
fig


Figure(layout=FigureLayout(height='420px'))